<a href="https://colab.research.google.com/github/chemineer/sudoku-generator/blob/master/my_sudoku.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# !/usr/bin/python
import sys
from functools import reduce
import random


###########
#Generator#
###########

class Generator:

    # constructor for generator, reads in a space delimited
    def __init__(self):
        
        numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 
                   4, 5, 6, 7, 8, 9, 1, 2, 3,
                   7, 8, 9, 1, 2, 3, 4, 5, 6, 
                   2, 1, 4, 3, 6, 5, 8, 9, 7, 
                   3, 6, 5, 8, 9, 7, 2, 1, 4, 
                   8, 9, 7, 2, 1, 4, 3, 6, 5, 
                   5, 3, 1, 6, 4, 2, 9, 7, 8, 
                   6, 4, 2, 9, 7, 8, 5, 3, 1, 
                   9, 7, 8, 5, 3, 1, 6, 4, 2]

        # constructing board
        self.board = Board(numbers)

    # function randomizes an existing complete puzzle
    def randomize(self, iterations):

        # not allowing transformations on a partial puzzle
        if len(self.board.get_used_cells())==81:

            # looping through iterations
            for x in range(0, iterations):

                # to get a random column/row
                case = random.randint(0, 4)
                

                # to get a random band/stack
                block = random.randint(0, 2) * 3
                

                # in order to select which row and column we shuffle an array of
                # indices and take both elements
                options = list(range(0,3))
                
                random.shuffle(options)
                piece1, piece2 = options[0],options[1]
                

                # pick case according to random to do transformation
                if case == 0:
                    
                    self.board.swap_row(block + piece1, block + piece2)
                    
                elif case == 1:
                    
                    self.board.swap_column(block + piece1, block + piece2)
                elif case == 2:
                    self.board.swap_stack(piece1, piece2)
                elif case == 3:
                    self.board.swap_band(piece1, piece2)
        else:
            raise Exception('Rearranging partial board may compromise uniqueness.')

    # method gets all possible values for a particular cell, if there is only one
    # then we can remove that cell
    def reduce_via_logical(self, cutoff = 81):
        cells = self.board.get_used_cells()
        random.shuffle(cells)
        for cell in cells:
                if len(self.board.get_possibles(cell)) == 1:
                    cell.value = 0
                    cutoff -= 1
                if cutoff == 0:
                    break

    # method attempts to remove a cell and checks that solution is still unique
    def reduce_via_random(self, cutoff=81):
        temp = self.board
        existing = temp.get_used_cells()

        # sorting used cells by density heuristic, highest to lowest
        new_set = [(x,self.board.get_density(x)) for x in existing]
        elements= [x[0] for x in sorted(new_set, key=lambda x: x[1], reverse=True)]

        # for each cell in sorted list
        for cell in elements:
            original = cell.value

            # get list of other values to try in its place
            complement = [x for x in range(1,10) if x != original]
            ambiguous = False

            # check each value in list of other possibilities to try
            for x in complement:

                # set cell to value
                cell.value = x

                # create instance of solver
                s = Solver(temp)

                # if solver can fill every box and the solution is valid then
                # puzzle becomes ambiguous after removing particular cell, so we can break out
                if s.solve() and s.is_valid():
                    cell.value = original
                    ambiguous = True
                    break

            # if every value was checked and puzzle remains unique, we can remove it
            if not ambiguous:
                cell.value = 0
                cutoff -= 1

            # if we ever meet the cutoff limit we can break out
            if cutoff == 0:
                break


    # returns current state of generator including number of empty cells and a representation
    # of the puzzle
    def get_current_state(self):
        template = "There are currently %d starting cells.\n\rCurrent puzzle state:\n\r\n\r%s\n\r"
        return template % (len(self.board.get_used_cells()),self.board.__str__())

#######
#Board#
#######

class Board:

    # initializing a board
    def __init__(self, numbers=None):

        # we keep list of cells and dictionaries to point to each cell
        # by various locations
        self.rows = {}
        self.columns = {}
        self.boxes = {}
        self.cells = []

        # looping rows
        for row in range(0, 9):
            # looping columns
            for col in range(0, 9):
                # calculating box
                box = 3 * (row / 3) + (col / 3)

                # creating cell instance
                cell = Cell(row, col, box)

                # if initial set is given, set cell value
                if not numbers is None:
                    cell.value = numbers.pop(0)
                else:
                    cell.value = 0

                # initializing dictionary keys and corresponding lists
                # if they are not initialized
                if not row in self.rows:
                    self.rows[row] = []
                if not col in self.columns:
                    self.columns[col] = []
                if not box in self.boxes:
                    self.boxes[box] = []

                # adding cells to each list
                self.rows[row].append(cell)
                self.columns[col].append(cell)
                self.boxes[box].append(cell)
                self.cells.append(cell)


    # returning cells in puzzle that are not set to zero
    def get_used_cells(self):
        return [x for x in self.cells if x.value != 0]

    # returning cells in puzzle that are set to zero
    def get_unused_cells(self):
        return [x for x in self.cells if x.value == 0]

    # returning all possible values that could be assigned to the
    # cell provided as argument
    def get_possibles(self, cell):
        all = self.rows[cell.row] + self.columns[cell.col] + self.boxes[cell.box]
        excluded = set([x.value for x in all if x.value!=0 and x.value != cell.value])
        results = [x for x in range(1, 10) if x not in excluded]
        return results

    # calculates the density of a specific cell's context
    def get_density(self,cell):
        all = self.rows[cell.row] + self.columns[cell.col] + self.boxes[cell.box]
        if cell.value != 0: all.remove(cell)
        return len([x for x in set(all) if x.value != 0])/20.0

    # gets complement of possibles, values that cell cannot be
    def get_excluded(self,cell):
        all = self.rows[cell.row] + self.columns[cell.col] + self.boxes[cell.box]
        excluded = set([x.value for x in all if x.value!=0 and x.value != cell.value])


    # swaps two rows
    def swap_row(self, row_index1, row_index2, allow=False):
        if allow or int(row_index1 / 3) == int(row_index2 / 3):
            for x in range(0, len(self.rows[row_index2])):
                temp = self.rows[row_index1][x].value
                self.rows[row_index1][x].value = self.rows[row_index2][x].value
                self.rows[row_index2][x].value = temp
        else:
            raise Exception('Tried to swap non-familial rows.')

    # swaps two columns
    def swap_column(self, col_index1, col_index2, allow=False):
        if allow or int(col_index1 / 3) == int(col_index2 / 3):
            for x in range(0, len(self.columns[col_index2])):
                temp = self.columns[col_index1][x].value
                self.columns[col_index1][x].value = self.columns[col_index2][x].value
                self.columns[col_index2][x].value = temp
        else:
            raise Exception('Tried to swap non-familial columns.')

    # swaps two stacks
    def swap_stack(self, stack_index1, stack_index2):
        for x in range(0, 3):
            self.swap_column(stack_index1 * 3 + x, stack_index2 * 3 + x, True)

    # swaps two bands
    def swap_band(self, band_index1, band_index2):
        for x in range(0, 3):
            self.swap_row(band_index1 * 3 + x, band_index2 * 3 + x, True)

    # copies the board
    def copy(self):
        b = Board()
        for row in range(0,len(self.rows)):
            for col in range(0,len(self.columns)):
                b.rows[row][col].value=self.rows[row][col].value
        return b

    # returns string representation
    def __str__(self):
        output = []
        for index, row in self.rows.items():
            my_set = list(map(str, [x.value for x in row]))
            new_set = []
            for x in my_set:
                if x == '0': new_set.append("_")
                else: new_set.append(x)
            output.append('|'.join(new_set))
        return '\r\n'.join(output)


    # exporting puzzle to a html table for prettier visualization
    def html(self):
        html = "<table>"
        for index, row in self.rows.items():
            values = []
            row_string = "<tr>"
            for x in row:
                if x.value == 0:
                    values.append(" ")
                    row_string += "<td>%s</td>"
                else:
                    values.append(x.value)
                    row_string +="<td>%d</td>"
            row_string += "</tr>"
            html += row_string % tuple(values)
        html += "</table>"
        return html

########
#Solver#
########

class Solver:

    # constructor for a solver, keeps a local copy of provided board
    def __init__(self, board):
        self.board = board.copy()
        self.vacants = self.board.get_unused_cells()

    # checks to make sure each compartment contains
    def is_valid(self):
        valid = set(range(1,10))
        for i,box in self.board.boxes.items():
            if not valid==set([x.value for x in box]):
                return False
        for i,row in self.board.rows.items():
            if not valid==set([x.value for x in row]):
                return False
        for i,col in self.board.columns.items():
            if not valid==set([x.value for x in col]):
                return False
        return True

    # solves a puzzle by moving forward and backwards through puzzle
    # and testing values
    def solve(self):
        index = 0
        while index>-1 and index < len(self.vacants):
            current = self.vacants[index]
            flag = False
            my_range = list(range(current.value+1,10))
            for x in my_range:
                if x in self.board.get_possibles(current):
                    current.value = x
                    flag = True
                    break
            if not flag:
                current.value = 0
                index -= 1
            else:
                index+=1
        if len(self.vacants)==0:
            return False
        else:
            return index == len(self.vacants)

######
#Cell#
######
class Cell:

    # defining the cell, each cell keeps track of its own value and location
    def __init__(self,row,col,box):
        self.row = row
        self.col = col
        self.box = box

        self.value = 0

    # returns a string representation of cell (for debugging)
    def __str__(self):
        temp = (self.value,self.row,self.col,self.box)
        return "Value: %d, Row: %d, Col: %d, Box: %d" % temp


#Main
# setting difficulties and their cutoffs for each solve method
difficulties = {
    'easy': (35, 0), 
    'medium': (81, 5), 
    'hard': (81, 10), 
    'extreme': (81, 15)
}

# getting desired difficulty from command line
difficulty = difficulties['hard']

# constructing generator object from puzzle file (space delimited columns, line delimited rows)
gen = Generator()

# applying 100 random transformations to puzzle
gen.randomize(100)

# getting a copy before slots are removed
initial = gen.board.copy()

# applying logical reduction with corresponding difficulty cutoff
gen.reduce_via_logical(difficulty[0])

# catching zero case
if difficulty[1] != 0:
    # applying random reduction with corresponding difficulty cutoff
    gen.reduce_via_random(difficulty[1])


# getting copy after reductions are completed
final = gen.board.copy()

# printing out complete board (solution)
print("The initial board before removals was: \r\n\r\n{0}".format(initial))

# printing out board after reduction
print("The generated board after removals was: \r\n\r\n{0}".format(final))


The initial board before removals was: 

6|2|4|8|7|9|5|3|1
9|8|7|1|3|5|6|4|2
5|1|3|2|4|6|9|7|8
1|3|2|6|5|4|7|8|9
4|6|5|9|8|7|1|2|3
7|9|8|3|2|1|4|5|6
2|4|1|5|6|3|8|9|7
8|7|9|4|1|2|3|6|5
3|5|6|7|9|8|2|1|4
The generated board after removals was: 

_|_|4|8|_|_|_|3|_
_|8|7|1|_|5|_|_|_
5|_|_|2|4|_|9|_|8
_|_|2|6|_|_|_|_|9
_|6|5|_|8|7|_|2|_
_|9|_|3|2|1|_|_|6
2|_|_|_|6|3|8|_|7
8|7|9|_|_|_|3|_|_
3|5|_|_|9|8|2|1|_
